In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import os
#mysql_jar_path = os.path.abspath("mysql-connector-j-9.0.0.jar")
#print("Caminho completo do JAR:", mysql_jar_path)


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

2025-05-17 00:07:16.166461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 00:07:16.430725: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 00:07:16.719977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747440436.858770    3419 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747440436.884773    3419 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747440437.083500    3419 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, sum



In [5]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf

In [4]:


ticker = "AMT"
data = yf.download(ticker,
                   start="2018-01-01", 
                   end="2020-12-31")    

print(data.head())



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open   Volume
Ticker             AMT         AMT         AMT         AMT      AMT
Date                                                               
2018-01-02  118.035690  119.783683  117.851690  119.733503  1880600
2018-01-03  118.495628  119.181451  117.809819  117.968730  1770500
2018-01-04  116.764412  118.729866  116.739322  118.294955  1729600
2018-01-05  117.517120  117.676031  116.747670  117.282940  2119400
2018-01-08  118.771645  119.800378  117.642565  117.642565  1882100


In [5]:
csv_AMT = f"{ticker}_AMT_historical_train_data_spark.csv"
data.to_csv(csv_AMT)

In [6]:
df_AMT= pd.read_csv("AMT_AMT_historical_train_data_spark.csv")

In [7]:
df_AMT.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMT,AMT,AMT,AMT,AMT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,118.03569030761719,119.78368259804581,117.85168977579708,119.73350295516077,1880600
3,2018-01-03,118.4956283569336,119.18145054894164,117.80981892678248,117.96872956941152,1770500
4,2018-01-04,116.76441192626953,118.72986625977379,116.73932210600027,118.29495477200356,1729600


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Parar sessão anterior, se necessário
try:
    spark.stop()
except:
    pass

# Criar nova SparkSession com o caminho exato do JAR
spark = SparkSession.builder \
    .appName("PySpark CSV to MySQL") \
    .master("local") \
    .config("spark.jars", "/usr/local/spark-3.4.4-bin-hadoop3/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

# Caminho do CSV
file_path = "file:///home/hduser/novo/AMT_AMT_historical_train_data_spark.csv"

# Leitura do CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Visualizar as primeiras 5 linhas
df.show(5)


25/05/17 00:10:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+----------+------------------+------------------+------------------+------------------+-------+
|     Price|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|    Ticker|               AMT|               AMT|               AMT|               AMT|    AMT|
|      Date|              null|              null|              null|              null|   null|
|2018-01-02|118.03569030761719|119.78368259804581|117.85168977579708|119.73350295516077|1880600|
|2018-01-03| 118.4956283569336|119.18145054894164|117.80981892678248|117.96872956941152|1770500|
|2018-01-04|116.76441192626953|118.72986625977379|116.73932210600027|118.29495477200356|1729600|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [9]:
# print the data type info
df.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [10]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+-------+
|     Price|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|    Ticker|               AMT|               AMT|               AMT|               AMT|    AMT|
|      Date|              null|              null|              null|              null|   null|
|2018-01-02|118.03569030761719|119.78368259804581|117.85168977579708|119.73350295516077|1880600|
|2018-01-03| 118.4956283569336|119.18145054894164|117.80981892678248|117.96872956941152|1770500|
|2018-01-04|116.76441192626953|118.72986625977379|116.73932210600027|118.29495477200356|1729600|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [11]:
# Remove the first two lines (CRL and null)
df = df.filter(df.Price != "Ticker").filter(df.Price != "Date")

In [12]:
df = df.toDF("Date", "Close", "High", "Low", "Open", "Volume")

In [13]:


df.show(5)



+----------+------------------+------------------+------------------+------------------+-------+
|      Date|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|2018-01-02|118.03569030761719|119.78368259804581|117.85168977579708|119.73350295516077|1880600|
|2018-01-03| 118.4956283569336|119.18145054894164|117.80981892678248|117.96872956941152|1770500|
|2018-01-04|116.76441192626953|118.72986625977379|116.73932210600027|118.29495477200356|1729600|
|2018-01-05|117.51712036132812|117.67603104845792| 116.7476695053117|117.28294022088228|2119400|
|2018-01-08|118.77164459228516|119.80037801502843|117.64256467527281|117.64256467527281|1882100|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [14]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [15]:


# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()



+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [16]:
# convert string double
df = df.withColumn("Close", col("Close").cast("double"))

In [17]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [18]:


df = df.withColumn("High", col("High").cast("double"))
df = df.withColumn("Low", col("Low").cast("double"))
df = df.withColumn("Open", col("Open").cast("double"))
df = df.withColumn("Volume", col("Volume").cast("double"))
df = df.withColumn("Date", col("Date").cast("Date"))



In [19]:


df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)



In [20]:
import mysql.connector
print(mysql.connector.__version__)

9.3.0


In [21]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+---------+
|      Date|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|2018-01-02|118.03569030761719|119.78368259804581|117.85168977579708|119.73350295516077|1880600.0|
|2018-01-03| 118.4956283569336|119.18145054894164|117.80981892678248|117.96872956941152|1770500.0|
|2018-01-04|116.76441192626953|118.72986625977379|116.73932210600027|118.29495477200356|1729600.0|
|2018-01-05|117.51712036132812|117.67603104845792| 116.7476695053117|117.28294022088228|2119400.0|
|2018-01-08|118.77164459228516|119.80037801502843|117.64256467527281|117.64256467527281|1882100.0|
+----------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [22]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [23]:
# MySQL connection properties
url = "jdbc:mysql://localhost:3306/companies"  # MySQL URL
properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [24]:
# Leitura do CSV
#df = spark.read.csv(file_path, header=True, inferSchema=True)

# Adiciona a coluna 'nome' com valor fixo "AAPL"
df_comp_nome = df.withColumn("nome", lit("AMT"))

# Escreve no banco MySQL
df_comp_nome.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/companies") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "AMT_historico") \
    .option("user", "root") \
    .option("password", "password") \
    .mode("overwrite") \
    .save()


In [25]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



mySQL

In [ ]:
import pandas as pd
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='companies'
)

df = pd.read_sql("SELECT * FROM AAPL_historico", conn)
df.to_csv("AAPL_historico_MYsql.csv", index=False)


In [ ]:
df_ABNB_Mysql= pd.read_csv("ABNB_historico_MYsql.csv")

In [ ]:
df_AAPL_Mysql.head()